In [1]:
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

from DDPG.ddpg import DDPG

args = {
    'gamma': 0.99,
    'render': True,
    'log_interval': 20
}

env = gym.make('BipedalWalker-v2')
episodes = 100000
print("action_space={}".format(env.action_space))
print("obs_space={}".format(env.observation_space))


def main():   
    task = {
        'state_size': 24,
        'action_size': 4,
        'action_high': 1,
        'action_low': -1
    }
    agent = DDPG(task)
    sum_reward = 0   
    for i_episode in range(episodes):
        running_reward = 0        
        state = env.reset()
        for t in range(10000):  # Don't infinite loop while learning
            action = agent.act(state, i_episode)                
            state, reward, done, _ = env.step(action)            
            if args['render']:
                env.render()                   
            running_reward += reward            
            if done:
                break
        
        sum_reward += running_reward
        
        if i_episode % args['log_interval'] == 0:
            avg_reward = sum_reward / args['log_interval']
            sum_reward = 0
            print('Episode {}\tLast length: {:5d}\t Reward: {:7.2f}\t Avg Reward: {:7.2f}'.format(
                i_episode, t, running_reward, avg_reward))
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break

print("threshold={} \n".format(env.spec.reward_threshold))
main()



Using TensorFlow backend.


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
action_space=Box(4,)
obs_space=Box(24,)
threshold=300 

Episode 0	Last length:   731	 Reward: -143.03	 Avg Reward:   -7.15
Episode 20	Last length:   287	 Reward: -129.52	 Avg Reward: -110.86
Episode 40	Last length:   661	 Reward: -136.48	 Avg Reward: -112.61
Episode 60	Last length:   169	 Reward: -100.64	 Avg Reward: -111.70
Episode 80	Last length:    71	 Reward: -103.18	 Avg Reward: -110.22
Episode 100	Last length:    46	 Reward: -102.24	 Avg Reward: -105.24
Episode 120	Last length:    83	 Reward: -121.83	 Avg Reward: -117.94
Episode 140	Last length:    52	 Reward: -116.26	 Avg Reward: -115.47
Episode 160	Last length:   144	 Reward: -121.31	 Avg Reward: -114.23
Episode 180	Last length:    74	 Reward:  -95.58	 Avg Reward: -110.36
Episode 200	Last length:    60	 Reward: -115.28	 Avg Reward: -

KeyboardInterrupt: 